In [17]:
offset = 0
limit = 3000
period = '5y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [18]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 8978


In [19]:
! pip install yfinance > /dev/null 2>&1
! mkdir hist

In [20]:
import yfinance as yf
import os, contextlib

In [21]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 2438
CPU times: user 1min 59s, sys: 2.81 s, total: 2min 2s
Wall time: 6min 34s


In [22]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

In [23]:
!mkdir stocks
!mkdir etfs

In [24]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [26]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        !mkdir {dest}/{s}
        shutil.move(join('hist', filename), join(dest,'{}'.format(s), 'data.csv'))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [27]:
! rmdir hist

In [33]:
# COPY TO S3
import boto3
s3C = boto3.client('s3')

def uploadDirectory(path,bucketname):
    for root,dirs,files in os.walk(path):
        for file in files:
            s3C.upload_file(os.path.join(root,file),bucketname,os.path.join(root,file))
            
uploadDirectory('etfs','carlosrodf')
uploadDirectory('stocks','carlosrodf')